<a href="https://colab.research.google.com/github/Delvys120/villages-property-tracker/blob/main/VLS_Expired_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


import os

folder_path = '/content/drive/MyDrive/VLS_Tracker'

# Check files in the folder
print("Files inside VLS_Tracker folder:")
print(os.listdir(folder_path))

import pandas as pd
import requests
from datetime import datetime

# Fetch today's data
print("🌐 Fetching today's VLS data...")
url = "https://api.thevillages.com/hf/search/allhomelisting"
response = requests.get(url)
data = response.json()

all_homes = data.get("HomeList", [])
print(f"✅ Total homes received: {len(all_homes)}")

# Filter PreOwned + Active listings
filtered_homes = [
    home for home in all_homes
    if home.get("SaleType") == "P" and home.get("ListingStatus") == "A"
]

print(f"🏡 Filtered PreOwned & Active homes: {len(filtered_homes)}")

# Save today's data
today = datetime.now().strftime('%Y-%m-%d')
today_filename = f'VLS_{today}.csv'
today_full_path = os.path.join(folder_path, today_filename)

df_today = pd.DataFrame([{
    "ULIKey": home.get("ULIKey"),
    "Address": home.get("Address"),
    "Village": home.get("Village"),
    "County": home.get("County"),
    "Model": home.get("Model"),
    "Price": home.get("Price").replace("$", "").replace(",", ""),
    "Bedrooms": home.get("Bedrooms"),
    "Baths": home.get("Baths"),
    "SquareFeet": home.get("SquareFeet"),
    "Garage": home.get("Garage"),
    "Pool": home.get("Pool"),
    "Latitude": home.get("GISLat"),
    "Longitude": home.get("GISLong"),
    "Status": home.get("ListingStatus"),
    "SaleType": home.get("SaleType"),
    "YouTubeVideoId": home.get("YouTubeVideoId"),
    "VLSNumber": home.get("VLSNumber")
} for home in filtered_homes])

df_today.to_csv(today_full_path, index=False, encoding='utf-8-sig')
print(f"💾 Today's listings saved as {today_full_path}")

from datetime import timedelta

# Get yesterday's date
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
yesterday_filename = f'VLS_{yesterday}.csv'
yesterday_full_path = os.path.join(folder_path, yesterday_filename)

# Check if the file exists
if not os.path.exists(yesterday_full_path):
    print(f"❌ Yesterday's file not found: {yesterday_filename}")
else:
    # Load the data
    df_yesterday = pd.read_csv(yesterday_full_path)
    print(f"✅ Yesterday's file loaded: {yesterday_filename}")

# Only Active listings from yesterday
df_yesterday_active = df_yesterday[df_yesterday['Status'] == 'A']

# Find homes missing today (removed)
removed = df_yesterday_active[~df_yesterday_active['ULIKey'].isin(df_today['ULIKey'])]

# Check if there are any removed listings
if not removed.empty:
    removed_filename = f'Removed_Listings_{today}.csv'
    removed_full_path = os.path.join(folder_path, removed_filename)

    # Save the removed listings to Google Drive
    removed.to_csv(removed_full_path, index=False, encoding="utf-8-sig")
    print(f"🗂️ Removed listings saved as {removed_full_path}")
else:
    print("✅ No removed listings today!")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files inside VLS_Tracker folder:
['VLS_2025-04-25.csv.csv', 'VLS_2025-04-26.csv', 'removed_Listings_2025-04-26.csv', 'VLS_2025-04-27.csv.csv', 'Removed_Listings_2025-04-27.csv', 'VLS_2025-04-27.csv']
🌐 Fetching today's VLS data...
✅ Total homes received: 1560
🏡 Filtered PreOwned & Active homes: 881
💾 Today's listings saved as /content/drive/MyDrive/VLS_Tracker/VLS_2025-04-27.csv
✅ Yesterday's file loaded: VLS_2025-04-26.csv
🗂️ Removed listings saved as /content/drive/MyDrive/VLS_Tracker/Removed_Listings_2025-04-27.csv
